In [1]:
%load_ext autoreload
%autoreload 2

import pathlib

import numpy as np
import matplotlib.pyplot as plt
import imageio

import torch
from torch import nn, optim
import torch.nn.functional as F

from core.layers import *
from core.utils import *
from core.models import *
torch.set_printoptions(sci_mode=False)

In [5]:
net_params, layer_infos = parse_darknet_cfg("cfg/yolov3-tiny.cfg")

In [6]:
net = Darknet(net_params, layer_infos)

In [11]:
net.load_weights("weights/yolov3-tiny.weights")

8858734
batch_norm here
batch_norm here
batch_norm here
batch_norm here
batch_norm here
batch_norm here
batch_norm here
batch_norm here
batch_norm here
no batch_norm here
batch_norm here
batch_norm here
no batch_norm here
8858734


In [6]:
detections = net(x)

In [12]:
detections[0, :21, :4] / 32

tensor([[ 0.4783,  0.5484,  0.8574,  2.3427],
        [ 0.5225,  0.4941,  5.0189,  3.8498],
        [ 0.5239,  0.5719,  6.4779,  9.4966],
        [ 0.4235,  1.6370,  2.1309,  4.0914],
        [ 0.4381,  1.3346,  2.8275,  2.3458],
        [ 0.2411,  1.5349, 12.9358, 16.3953],
        [ 0.3281,  2.6609,  1.9467,  1.5835],
        [ 0.5914,  2.2955,  4.9249,  5.0984],
        [ 0.2744,  2.5458, 16.5545, 21.8439],
        [ 0.3733,  3.6103,  5.1404,  1.4062],
        [ 0.6628,  3.5368,  2.9110,  2.8516],
        [ 0.2020,  3.4422,  9.1731, 15.5886],
        [ 0.4218,  4.6056,  1.6285,  1.8431],
        [ 0.5725,  4.2555,  4.0777,  2.9440],
        [ 0.5621,  4.5854,  5.6253,  7.9249],
        [ 0.4136,  5.6826,  0.8587,  1.4887],
        [ 0.5085,  5.6261,  2.8444,  2.7643],
        [ 0.4336,  5.7645, 15.5707,  7.9424],
        [ 0.6809,  6.3955,  2.1564,  3.0839],
        [ 0.5447,  6.5677,  1.6626,  6.0714],
        [ 0.5488,  6.6893, 10.4614, 13.4394]], grad_fn=<DivBackward0>)

In [24]:
module_list = get_layers(net_params, layer_infos)

In [39]:
weight_path = "weights/yolov3-tiny.weights"

             

In [68]:
a =np.random.randn(100)
a

array([-0.91183471, -1.98457233,  0.22166992, -0.4135193 , -0.3260484 ,
       -0.27250718, -0.73718413,  0.73476597, -2.71321912,  0.76695162,
       -0.82399737,  0.58645394,  0.56753388,  0.73581492, -0.19255175,
        0.28950051, -0.65010059,  0.38207499, -0.1593287 , -0.59359104,
        0.32452083,  0.01254573, -0.49072232,  0.80269107,  0.40856557,
        0.50302949, -0.24444209,  0.37791295, -0.26664745, -1.46128504,
        0.23326992,  0.02594722, -0.24125236, -0.70920601, -0.35684534,
       -1.84135099,  1.19903287, -0.23538413, -0.6250458 ,  0.28331517,
       -0.95826191,  0.32623083, -0.23541449,  0.60792159,  0.81297471,
        0.45569041, -0.77034322, -0.66719094,  1.14997418, -0.08367423,
       -1.69414651, -0.61493023,  0.29588771,  0.00401442,  0.63989825,
        0.52091433,  0.29810576,  0.2711127 ,  0.3271997 ,  0.25368677,
       -0.25763093, -0.88943164,  1.08185021,  1.39688086,  0.38195897,
        0.58521092, -2.74533801, -0.35288031,  0.4941963 ,  1.34

In [63]:
bn = nn.BatchNorm2d(20)

In [80]:
_load_params(bn.running_mean, a, 1)

21

In [82]:
bn.running_mean

tensor([-1.9846,  0.2217, -0.4135, -0.3260, -0.2725, -0.7372,  0.7348, -2.7132,
         0.7670, -0.8240,  0.5865,  0.5675,  0.7358, -0.1926,  0.2895, -0.6501,
         0.3821, -0.1593, -0.5936,  0.3245])